In [1]:
import os
import glob
from osgeo import gdal
from tqdm import tqdm
import cv2
import shutil
import pandas as pd
import numpy as np
from skimage.morphology import square, dilation
from matplotlib import pyplot as plt
import geopandas as gpd
from PIL import Image

In [2]:
root= "F:/Berlin-DOP_2020/Lidar/Highest raster"  #ground truth dataset with tiles of res 10000*10000
images_out= "F:/Berlin-DOP_2020/Lidar/Highest raster/Split" #output location for splitted images

In [3]:
def geo_tile(untiled_image_dir, tiles_out_dir, tile_size=500,
             overlap=0.0, search=".png",Output_Channels=[1,2,3]):
    """Function to tile a set of images into smaller square chunks with embedded georeferencing info
    allowing an end user to specify the size of the tile, the overlap of each tile, and when to discard
    a tile if it contains blank data.
    Arguments
    ---------
    untiled_image_dir : str
        Directory containing full or partial image strips that are untiled.
        Imagery must be georeferenced.
    tiles_out_dir : str
        Output directory for tiled imagery.
    tile_size : int
        Extent of each tile in both X and Y directions in units of pixels.
        Defaults to ``544`` .
    overlap : float
        The amount of overlap of each tile in float format.  Should range between 0 and <1.
        Defaults to ``0.2`` .
    search : str
        A string with a wildcard to search for files by type
        Defaults to ".png"
    Output_Channels : list
        A list of the number of channels to output, 1 indexed.
        Defaults to ``[1,2,3]`` .
    Returns
    -------
    Tiled imagery directly output to the tiles_out_dir
    """
    if not os.path.exists(tiles_out_dir):
        os.makedirs(tiles_out_dir)
    
    os.chdir(untiled_image_dir)
    search2 = "*" + search
    images = glob.glob(search2)
    tile_size = int(tile_size)

    for stackclip in images:
        print(stackclip)
        interp = gdal.Open(os.path.abspath(stackclip))
        width = int(interp.RasterXSize)
        height = int(interp.RasterYSize)
        count = 0
        for i in range(0, width, int(tile_size * (1 - overlap))):
            for j in range(0, height, int(tile_size * (1 - overlap))):
                Chip = [i, j, tile_size, tile_size]
                count += 1
                # Adding the name according to the longitude and latitude values (EPSG:25833)
                name_x = (int(stackclip.split('_')[1])*1000 + (i/500)*50) 
                name_y = (int(stackclip.split('_')[2])*100 + ((9500-j)/500)*5)*10

#                 Tileout = tiles_out_dir + "/" + \
#                     stackclip.split(search)[0] + "_tile_" + str(count) + ".jpeg"
                Tileout = tiles_out_dir + "/" + \
                str(stackclip.split('_')[0]) + "_" + str(name_x) + "_"+ str(name_y) +"_tile_" + str(count) + ".jpeg"
                output = gdal.Translate(Tileout, stackclip, srcWin=Chip, bandList=Output_Channels)
                del output
    print("Done")

In [4]:
os.chdir(root)
dirs=glob.glob(f"{root}/")

#print(dirs)
for directory in dirs:
    s = directory.replace('\\','/')
    print(s)
    geo_tile(s, images_out, tile_size=500, overlap=0.0,search="*.png",Output_Channels=[1,2,3]) #No overlap for testing.
    

F:/Berlin-DOP_2020/Lidar/Highest raster/
Maskeddop10rgbi_391_5821_1_be_2020.png
Maskeddop10rgbi_391_5822_1_be_2020.png
Maskeddop10rgbi_391_5823_1_be_2020.png
Maskeddop10rgbi_391_5824_1_be_2020.png
Maskeddop10rgbi_392_5821_1_be_2020.png
Maskeddop10rgbi_392_5822_1_be_2020.png
Maskeddop10rgbi_392_5823_1_be_2020.png
Maskeddop10rgbi_392_5824_1_be_2020.png
Done
